### HW1 with resnet50 pretrained model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import os
from torch.utils.data import DataLoader
import time
import copy




In [2]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [3]:
# Done as if they are grayscale

data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.Resize((256, 256)),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for grayscale
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ]),
}


In [4]:
# Define dataset paths
data_dir = './covid_ct/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True)
               for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

In [5]:
# Use resnet50
model_ft = models.resnet50(weights='IMAGENET1K_V1')

# Modify the first convolutional layer to accept 1-channel input
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Freeze all layers for feature extraction
for param in model_ft.parameters():
    param.requires_grad = False

# Replace the final fully connected layer for 2 outputs
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)  # 2 classes: COVID/Normal

model_ft = model_ft.to(device)  # Move to GPU


# Define loss function 
criterion = nn.CrossEntropyLoss()

# Define the optimizer to only update the final layer's parameters
optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)


# Learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                # Move inputs and labels to GPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model




In [7]:
# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

# Save the model
torch.save(model_ft.state_dict(), 'resnet50_covid_transfer_learning.pth')

Epoch 0/0
----------
train Loss: 0.4797 Acc: 0.7750
test Loss: 0.3855 Acc: 0.8443

Training complete in 2m 8s
Best test Acc: 0.8443


#### Regarding data augmentation

<u>Transforms which are ok:</u>

transforms.Resize(256)
- OK: Resizing is fine, but ensure the aspect ratio is preserved to avoid distorting anatomical structures.
Tip: Use transforms.Resize((256, 256)) to force a square shape if needed.

transforms.CenterCrop(224)
- OK: Center cropping is common, but ensure the region of interest (e.g., lungs) is not cropped out.
- Tip: If the lungs are not centered, consider using a custom crop or segmentation mask.

transforms.ToTensor()
- OK: Required to convert images to PyTorch tensors.

<u>Transforms not to have:</u>

transforms.RandomHorizontalFlip()
- Not ideal for CT scans: Flipping a CT scan horizontally can reverse anatomical structures (e.g., heart position), which may not be realistic for medical data.
- Alternative: Use vertical flips (if clinically meaningful) or rotations (e.g., ±10°) to augment data without reversing anatomy.

transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
- Not ideal for CT scans: These mean/std values are for ImageNet (RGB images). CT scans are grayscale and have a different intensity range (e.g., Hounsfield Units, typically -1000 to +1000).
- Alternative: Calculate the mean and std from your CT dataset, or use a fixed range (e.g., normalize to [0, 1] or [-1, 1]).
Example: